Evaluating the model trained with **Sentinel-2 L2A** data (natural-looking images).

# 0. Setup

## 0.1. Libraries

In [ ]:
# Utilities
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Deep learning
import torch
from torch import nn, utils
from torchmetrics import classification

# Custom library
from library import nn_model, utilities, visualizations, preprocessing

# Device
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device = torch.device(device)
print(f'Using device: {device}')

# Utilities
# classes = {0:'non-building', 255:'building'}
seed = 42  # Set the same seed as for the corresponding training run with S2-L2A data!
utilities.set_seed(seed)

Using device: cuda
Seed set to 42 for NumPy, Torch and Random for reproducibility.


## 0.2. Setting the path to the data

We set the path to the tiles that have been created in the preprocessing notebook, `3_preprocessing.ipynb`.

In [ ]:
base_input_dir = '/media/pablo/Shared files/data/'  # Adjust this path to your data directory containing the labelled dataset
input_labelled_dir = os.path.join(base_input_dir, 'Satellite_burned_area_dataset')  # Path to the original labelled dataset
tile_dir = os.path.join(base_input_dir, 'tiled_labelled_dataset')  # Path to the tiled dataset
sentinel_type = 2  # Sentinel-2 data

## 0.3. Notebook description

In this notebook we evaluate the model trained on Sentinel-2 L2A images on the test split made in that notebook.

> For consistency, in this notebook we again create the same split in the same way, but in this case we will only be using the test data. **Please ensure that the seeds set in both notebooks are the same for avoiding data leakage and creating the same test set in both notebooks**. Furthermore, pre-processing must be implemented in the same way as for the training notebook with Sentinel-2 L2A images. It is done here again for clarity.

*Reminder of label interpretation: **the brighter the label in the mask, the higher the severity of the fire (white if maximum severity)**. Areas affected by fires of lower severity are darker. There are 5 severity levels, from 0 (not burned) to 4 (maximum burn damage).*

# 1. Preparing the data

## 1.1. Getting the image and masks paths

In [ ]:
# Get all of the folders within the tiles directory
fire_folders = sorted([f for f in os.listdir(tile_dir) if os.path.isdir(os.path.join(tile_dir, f))])
# Print the first 5 folders
print("First 5 fire folders:")
print(fire_folders[:5])
# Print the total number of fire folders
print("Total number of fire folders:", len(fire_folders))

# Get the paths for the images and the labels
image_paths = []
label_paths = []
for fire_folder in fire_folders:
    print(f"Fire folder: {fire_folder}")
    images_path = os.path.join(tile_dir, fire_folder, 'images')
    labels_path = os.path.join(tile_dir, fire_folder, 'masks')
    if os.path.exists(images_path):
        image_paths.append(sorted([os.path.join(images_path, img) for img in os.listdir(images_path) if img.startswith(f'sentinel{sentinel_type}') and img.endswith('.tiff')]))
    else:
        print(f"Images path does not exist: {images_path}")
    if os.path.exists(labels_path):
        label_paths.append(sorted([os.path.join(labels_path, lbl) for lbl in os.listdir(labels_path) if lbl.endswith('.tiff')]))
    else:
        print(f"Labels path does not exist: {labels_path}")
# Print the first 5 image paths
print(f"First 5 image and label paths of the fire folder: {fire_folders[0]}")
print(image_paths[0][:5])  # The result is a list of lists, where each sublist contains the paths of images for a specific fire folder
print(label_paths[0][:5])  # The result is a list of lists, where each sublist contains the paths of labels for a specific fire folder

# Print the total number of lists in image_paths and label_paths (should be equal to the number of fire folders)
print("Total number of fire folders in image_paths:", len(image_paths))
print("Total number of fire folders in label_paths:", len(label_paths))

First 5 fire folders:
['EMSR207_01MIRANDADOCORVO_02GRADING_MAP_v2_vector', 'EMSR207_02LOUSA_02GRADING_MAP_v2_vector', 'EMSR207_03PAMPILHOSADASERRA_02GRADING_MAP_v2_vector', 'EMSR207_04AVELAR_02GRADING_MAP_v2_vector', 'EMSR207_05PEDROGAOGRANDE_02GRADING_MAP_v2_vector']
Total number of fire folders: 73
Fire folder: EMSR207_01MIRANDADOCORVO_02GRADING_MAP_v2_vector
Fire folder: EMSR207_02LOUSA_02GRADING_MAP_v2_vector
Fire folder: EMSR207_03PAMPILHOSADASERRA_02GRADING_MAP_v2_vector
Fire folder: EMSR207_04AVELAR_02GRADING_MAP_v2_vector
Fire folder: EMSR207_05PEDROGAOGRANDE_02GRADING_MAP_v2_vector
Fire folder: EMSR207_06MADEIRA_02GRADING_MAP_v2_vector
Fire folder: EMSR207_07ALVAIAZERE_02GRADING_MAP_v2_vector
Fire folder: EMSR207_08CERNACHEDOBONJARDIM_02GRADING_MAP_v2_vector
Fire folder: EMSR207_10ORVALHO_02GRADING_MAP_v2_vector
Fire folder: EMSR209_01MOGUER_02GRADING_MAP_v2_vector
Fire folder: EMSR209_02MAZAGON_02GRADING_MAP_v2_vector
Fire folder: EMSR209_03LOSCABEZUDOS_02GRADING_MAP_v1_vecto

## 1.2. Loading the image and masks into tensors

We load the images and labels as `torch.Tensor` of dimensions $n \times d \times h \times w$, where:
- $n$: number of images (tiles)/masks.
- $d$: number of channels for the images/masks.
- $h$: height of the images and of the masks. Both are of 256 height, as designed in `3_preprocessing`.
- $w$: width of the images and of the masks. Both are of 256 height, as designed in `3_preprocessing`.

We normalize each channel to $N[0,1]$ and set the data type as `torch.float32` (set to `torch.float16` for greater memory efficiency). Why we normalize image channels:

1. **Gradient stability**: Different channels may have vastly different value ranges, leading to unstable gradients
2. **Training speed**: Normalized inputs help the optimizer converge faster
3. **Weight initialization**: Most weight initialization schemes assume normalized inputs
4. **Activation functions**: Work optimally with inputs in specific ranges

In [ ]:
# Flatten the lists of image and label paths
images_flat = [item for sublist in image_paths for item in sublist]
labels_flat = [item for sublist in label_paths for item in sublist]

# Read images and labels and stack them into a single tensor (reduce precision to float16 for greater efficiency)
images = torch.stack([utilities.read_tiff_to_torch(file_path = image, dtype=torch.float32, normalize=True, normalization_method='per_channel') for image in images_flat])
labels_raw = torch.stack([utilities.read_tiff_to_torch(file_path = label, dtype = torch.float32, normalize=True, normalization_method='255') for label in labels_flat])  # Masks are encoded in 0-255 range according to the paper

Now, let's check the array dimensions and display several image–label pairs to ensure that they are correctly aligned.

In [ ]:
print('Size of the image tensor:', images.size())
print('Size of the label tensor:', labels_raw.size())

# Check if the number of images and labels are equal
if images.size(0) != labels_raw.size(0):
    raise ValueError('Number of images and labels do not match!')

Size of the image tensor: torch.Size([3230, 13, 256, 256])
Size of the label tensor: torch.Size([3230, 1, 256, 256])


i.e., in this case:

- $n$: we have 3,230 images and masks (3,374 images before removing those without coverage).
- $d$: number of channels for the images/masks. The images have 13 dimensions (bands B01-B12 from Sentinel-2 L2A, https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S2L2A.html#available-bands-and-data).
- $h$: height of the images and of the masks. Both are of 256 height, as designed in `3_preprocessing`.
- $w$: width of the images and of the masks. Both are of 256 height, as designed in `3_preprocessing`.

## 1.3. Mapping the classes to integers 

In [ ]:
# Apply the mapping
labels = utilities.map_labels_to_classes_approximate(labels_raw)

del labels_raw  # Free up memory

In [ ]:
# Print the unique values in the labels tensor to verify the mapping
print("Unique values in the labels tensor after mapping:", torch.unique(labels))

Unique values in the labels tensor after mapping: tensor([0, 1, 2, 3, 4])


## 1.4. Formatting the data as a `TensorDataset`

Useful documentation: [`torch.utils.data`](https://pytorch.org/docs/stable/data.html)

In [ ]:
# Format the data as a TensorDataset
dataset = utils.data.TensorDataset(images, labels)  # Create a TensorDataset from the image and label tensors
print(len(dataset))

3230


## 1.5. Splitting data into train, validation and test

We split into 70% training data, 15% validation and 15% test.

> Ensure that the seed is set to the same number as the model that is going to be used for prediction (check in the filename).

In [ ]:
# Split the dataset into training, validation and test sets
train_dataset, val_dataset, test_dataset = utils.data.random_split(dataset, [0.7, 0.15, 0.15], generator=torch.Generator().manual_seed(seed))

# Set batch size
batch_size = 64

# Set number of physical CPU cores (as int) to use for data loading
num_workers = os.cpu_count() // 2 if os.cpu_count() is not None else 0

# Create DataLoader just for the test dataset
test_loader = utils.data.DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False,  # Keep test data in the same order for consistent evaluation
    num_workers=num_workers,  # Use multiple workers for faster data loading
    pin_memory=True if device == 'cuda' else False,  # Pin memory for faster data transfer to GPU
)

# 2. Error analysis with the test set

## 2.1. Loading a saved PyTorch model

Check the `models` directory within this repo. In principle, opt for the models with the lowest validation loss.

To load a save model, we need to:
1. Initialize a new model with the same architecture
2. Load the state dictionary from the saved file (in the section "Estimating the model weights", this is done in the line `torch.save(model.state_dict(), [...])`)
3. Apply the state dictionary to the model

All of these steps are automatically implemented with the `load_model` function of the `library`.

In [ ]:
model_name = 'unet_sentinel2_seed42_epochs_200_val_loss_0.9021.pth'

model = utilities.load_model(os.path.join('models', model_name))

## 2.2. Predict probabilities for test images

For more information, check the `predict` function in the `library`. It follows the same logic as the `train_model` function, but is optimized for inference.

In [ ]:
# Make predictions on the test set
test_predictions, test_labels = nn_model.predict(
    model=model,
    dataloader=test_loader,
    device=device,
    num_classes=5,  # Number of classes in the dataset (0-4 for none to high burn severity)
    return_probs=False,  # Get predictions as class labels
)

print(f"Predictions shape: {test_predictions.shape}")
print(f"Labels shape: {test_labels.shape}")

## 2.3. Visualizing sample images and their predictions

For more information, check `visualize_predictions` in `library`.

> THE `visualize_predictions` FUNCTION MUST BE ADAPTED TO THESE MULTI-CHANNEL IMAGES!!! TAKE `display_image` AS THE INSPIRATION (OR EVEN CALL IT WITHIN THE `visualize_predictions` FUNCTION). I have already made some changes that should make the function work, but I'm not 100% sure.

In [ ]:
# Get a batch of test images and their labels
test_images_sample = []
test_true_masks_sample = []

# Extract a few batches from the test loader
for images, labels in test_loader:
    test_images_sample.append(images)
    test_true_masks_sample.append(labels.squeeze(1))
    if len(test_images_sample) >= 3:  # Get 3 batches at most
        break

# Concatenate batches
test_images_sample = torch.cat(test_images_sample, dim=0)
test_true_masks_sample = torch.cat(test_true_masks_sample, dim=0)

# Make predictions (just for 3 batches)
with torch.no_grad():
    model.eval()
    # Get raw logits from the model [N, C, H, W]
    test_pred_logits = model(test_images_sample.to(device)).cpu()
    # Get predicted class for each pixel by taking the argmax along the class dimension
    test_pred_masks_sample = torch.argmax(test_pred_logits, dim=1)

# Visualize
visualizations.visualize_predictions(
    images=test_images_sample, 
    true_masks=test_true_masks_sample, 
    predictions=test_pred_masks_sample, 
    num_samples=3,
    num_classes=5, # As defined in the notebook
    rgb_bands=(3, 2, 1) # Sentinel-2 RGB bands are B4, B3, B2 which are indices 3, 2, 1
)

## 2.4. Classification metrics

We use several classification metrics to assess the performance of the model, including accuracy, precision, recall, F1 and intersection over union (IoU), as well as the confusion metrics for multi-class classification.

### Confusion matrix

In [ ]:
# Flatten the predictions and labels to 1D arrays for confusion matrix calculation
y_pred_flat = test_predictions.flatten()
y_true_flat = test_labels.flatten()

# Convert to numpy arrays if they're not already
if torch.is_tensor(y_pred_flat):
    y_pred_flat = y_pred_flat.cpu().numpy()
    y_true_flat = y_true_flat.cpu().numpy()

# Define class names for the plot
class_names = ['Unburned', 'Low Sev.', 'Moderate-Low Sev.', 'Moderate-High Sev.', 'High Sev.']

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true_flat, y_pred_flat)

# Visualization of confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names,
            yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Burn Severity')
plt.show()

### Full classification report

In [ ]:
# Calculate IoU
jaccard_metric = classification.MulticlassJaccardIndex().to(device)  # Initialize metrics
# Move tensors to the same device as the metrics to avoid errors
test_predictions_device = test_predictions.to(device)
test_labels_device = test_labels.to(device)
# Calculate metrics (make sure inputs are tensors)
iou_torch = jaccard_metric(test_predictions_device, test_labels_device)

# Define class names for the report
class_names = ['Unburned (0)', 'Low Sev. (1)', 'Mod-Low Sev. (2)', 'Mod-High Sev. (3)', 'High Sev. (4)']

print(classification_report(y_true_flat, y_pred_flat, 
                           target_names=class_names))

### Precision-recall for multi-class?

### ROC curve for multi-class?

### Insights

## 2.5. Visualization of classification errors

Let's visualize true positives, true negatives, false positives, and false negatives as overlays on the input image.

For more information, check `visualize_segmentation_errors` and `visualize_segmentation_errors_sample` (for visualizing segmentation errors of just a sample) in `library`.

> THE FUNCTIONS MUST BE ADAPTED TO THESE MULTI-CHANNEL IMAGES!!! TAKE `display_image` AS THE INSPIRATION. I have already made some changes that should make the function work, but I'm not 100% sure.

In [ ]:
# Visualize a few random samples
visualizations.visualize_segmentation_errors_sample(
    images=test_images_sample,
    true_masks=test_true_masks_sample,
    predictions=test_pred_masks_sample,
    num_classes=5,
    num_samples=3,
    rgb_bands=(3, 2, 1) # S2 RGB bands are B4, B3, B2
)

## 2.5. Setting the optimal classification threshold

Assuming that we give equal importance to false positives and false negatives, we set a decision probability threshold that strikes a good balance between the two.

Since we want to give equal importance to reduce false positives (related to precision, as it is the ratio of positive instances correctly predicted over all positive predictions) and false negatives (related to recall, since it is the ratio of positive instances correctly predicted over all positive instances), **we should probably aim for a decision probability threshold which maximizes the F1-score, since it is the harmonic mean of precision and recall**, where:

$$F1-score = \frac{2\times Precision \times Recall}{Precision + Recall}$$

To do that, it can be useful to start by plotting the precision-recall curve for different probability thresholds (just to visualize the trade-off when changing the probability threshold), and then search for the probability threshold which maximizes the F1-score.

### Precision-recall curve with F1-score for different probability thresholds

In [ ]:
# Make predictions on the test set
test_predictions_prob, test_labels = nn_model.predict(
    model=model,
    dataloader=test_loader,
    device=device,
    num_classes=5,  # Number of classes in the dataset (0-4 for none to high burn severity)
    return_probs=True,  # Get probabilities instead of predictions of the most likely class
)

print(f"Predictions shape: {test_predictions.shape}")
print(f"Labels shape: {test_labels.shape}")

In [ ]:
# Third, we initialize the metrics and lists for storing the results
precision_metric = classification.BinaryPrecision().to(device)
recall_metric = classification.BinaryRecall().to(device)
f1_metric = classification.BinaryF1Score().to(device)

results = {
    'thresholds': [],
    'precision': [],
    'recall': [],
    'f1_score': []
}

# Fourth, we calculate the metrics for different thresholds 
for threshold in np.arange(0, 1.01, 0.005):

    # Move tensors to the same device as the metrics to avoid errors
    test_predictions_device = test_predictions_prob.to(device)
    test_labels_device = test_labels.to(device)

    # Convert probabilities to binary predictions using the threshold
    test_predictions_binary = (test_predictions_device > threshold).float()

    # Calculate metrics (make sure inputs are tensors)
    precision_torch = precision_metric(test_predictions_binary, test_labels_device)
    recall_torch = recall_metric(test_predictions_binary, test_labels_device)
    f1_torch = f1_metric(test_predictions_binary, test_labels_device)

    # Store the results
    results['thresholds'].append(threshold)
    results['precision'].append(precision_torch.item())
    results['recall'].append(recall_torch.item())
    results['f1_score'].append(f1_torch.item())

In [ ]:
# Now, we convert the results to a DataFrame for better visualization
results_df = pd.DataFrame(results)

results_df.head()

In [ ]:
# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(results_df['recall'], results_df['precision'])
plt.title('Precision-recall curve with F1-scores')

# Add vertical and horizontal lines for precision and recall of maximum F1-score
max_f1_index = results_df['f1_score'].idxmax()
plt.scatter(results_df['recall'][max_f1_index], results_df['precision'][max_f1_index], 
            color='red', s=100, zorder=5, label=f'Best Threshold (F1 = {results_df['f1_score'][max_f1_index]:.2f})')

# Adjust scale of the axes
plt.xlim(0, 1)
plt.ylim(0, 1)

# Add grid, labels and legend
plt.grid()
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()

plt.show()

- The beginning of the curve represents the highest probability thresholds possible.
- On the other hand, the end of the curve represents the lowest probability thresholds possible, where the recall converges to a high value (as the number of false negatives will be very low, as we will more easily classify an instance as positive).
- The precision converges, at the end of the curve, to the class imbalance proportion, where we will classify all instances as positive. This happens because all the actual positives will be correctly predicted, but all the actual negatives will be incorrectly predicted.

### Optimal classification threshold

In [ ]:
# Print the best threshold in terms of the F1-score
print('Best threshold and metrics for the maximum F1-score:')
display(results_df.loc[results_df['f1_score'].idxmax()])

# 3. Conclusions